In [1]:
from data import import_dataset, load_dataset, FORUM, PROLIFIC, DATASETS
dataset = DATASETS


In [2]:
import_dataset(dataset)

anno
anno
prolific
prolific
prolific


In [3]:
ds=load_dataset(dataset)

In [4]:
d = ds.to_df()

In [5]:
from sklearn.model_selection import train_test_split

text_train, text_test, labels_train, labels_test = train_test_split(d["text"], d["self"], test_size=0.2, stratify=d["source"])

pandas.core.series.Series

In [7]:
from classifiers.sner import SNERConfig, create_train_file, create_config_file, train_sner

config = SNERConfig()

create_config_file(config)
create_train_file(labels_train)

#train_sner()